# Iteration 4: warm-up period stage 2

## Plain English Summary

This is a non-terminating simulation.  The next step is to implement a warm-up period parameter for the model.  This is user settable.




## Notebook imports

In [3]:
from IPython.display import Markdown, display

import numpy as np
import math

## LLM Prompt

In [4]:
display(Markdown("../04_prompts_stage1/inputs/iteration_4.md"))

modify the code as follows

the model should include a user settable warm up period. This defaults to 1 month.

model run length should be renamed results collection period and have the default value of 12 months.  

The total run length of the model is the warm up period plus the results collection period.

the Experiment class should also accept a parameter for the warm up period

Show all of the model code: this includes all patient generator functions in the CCModel class and all code in the Experiment class.

## Date and time of input to LLM

> 1216 05th June 2024 

## Observations on the LLM Output


* The warm-up period has been added to `Experiment` as a parameter.
* `run_length` has been correctly renamed to `results_collection_period`
* The model is run for the `results_collection_period` + `warm_up_period` saved as a
  new attribute called `total_run_length`
* At the moment the warm-up period is not used anywhere else in the model.

## LLM Output 1 stage 2

The changes made are:
In the Experiment class, the run_length parameter is renamed to results_collection_period with a default value of 12 * 30 * 24 (12 months in hours).
A new parameter warm_up_period is added to the Experiment class with a default value of 30 * 24 (1 month in hours).
A new attribute total_run_length is calculated as the sum of warm_up_period and results_collection_period.
In the run method of the CCU class, the simulation is run until self.experiment.total_run_length.
All patient generator functions (accident_emergency_arrivals, wards_arrivals, emergency_surgery_arrivals, other_hospitals_arrivals, xray_department_arrivals, and elective_surgery_arrivals) and the Experiment class are shown in the modified code.
The previous changes made to the model, including incrementing the patient_id_counter and providing the arrival print statements after using yield in the generator functions, and adding the elective_surgery_arrival_std_dev parameter to the Experiment class, are also included in this modified code

In [5]:
import simpy
import numpy as np
from scipy.stats import lognorm

class Experiment:
    def __init__(self, accident_emergency_arrival_rate=22.72, wards_arrival_rate=26.0, emergency_surgery_arrival_rate=37.0, other_hospitals_arrival_rate=47.2, xray_department_arrival_rate=575.0, elective_surgery_arrival_rate=17.91, elective_surgery_arrival_std_dev=3.16, accident_emergency_lognormal_mu=128.79, accident_emergency_lognormal_sigma=267.51, wards_lognormal_mu=177.89, wards_lognormal_sigma=276.54, emergency_surgery_lognormal_mu=140.15, emergency_surgery_lognormal_sigma=218.02, other_hospitals_lognormal_mu=212.86, other_hospitals_lognormal_sigma=457.67, xray_department_lognormal_mu=87.53, xray_department_lognormal_sigma=108.67, elective_surgery_treatment_rate=57.34, num_critical_care_beds=24, intensive_cleaning_duration=5, results_collection_period=12 * 30 * 24, warm_up_period=30 * 24):
        self.accident_emergency_arrival_rate = accident_emergency_arrival_rate
        self.wards_arrival_rate = wards_arrival_rate
        self.emergency_surgery_arrival_rate = emergency_surgery_arrival_rate
        self.other_hospitals_arrival_rate = other_hospitals_arrival_rate
        self.xray_department_arrival_rate = xray_department_arrival_rate
        self.elective_surgery_arrival_rate = elective_surgery_arrival_rate
        self.elective_surgery_arrival_std_dev = elective_surgery_arrival_std_dev
        self.accident_emergency_lognormal_mu = accident_emergency_lognormal_mu
        self.accident_emergency_lognormal_sigma = accident_emergency_lognormal_sigma
        self.wards_lognormal_mu = wards_lognormal_mu
        self.wards_lognormal_sigma = wards_lognormal_sigma
        self.emergency_surgery_lognormal_mu = emergency_surgery_lognormal_mu
        self.emergency_surgery_lognormal_sigma = emergency_surgery_lognormal_sigma
        self.other_hospitals_lognormal_mu = other_hospitals_lognormal_mu
        self.other_hospitals_lognormal_sigma = other_hospitals_lognormal_sigma
        self.xray_department_lognormal_mu = xray_department_lognormal_mu
        self.xray_department_lognormal_sigma = xray_department_lognormal_sigma
        self.elective_surgery_treatment_rate = elective_surgery_treatment_rate
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.results_collection_period = results_collection_period
        self.warm_up_period = warm_up_period
        self.total_run_length = self.warm_up_period + self.results_collection_period

class CCU:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_id_counter = 0
        self.cancelled_operations = 0
        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)
        self.rng_elective_surgery = np.random.default_rng(seed=47)
        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=self.experiment.num_critical_care_beds)

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(self.experiment.accident_emergency_arrival_rate))
            print(f"Patient {self.patient_id_counter} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(self.experiment.wards_arrival_rate))
            print(f"Patient {self.patient_id_counter} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(self.experiment.emergency_surgery_arrival_rate))
            print(f"Patient {self.patient_id_counter} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(self.experiment.other_hospitals_arrival_rate))
            print(f"Patient {self.patient_id_counter} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(self.experiment.xray_department_arrival_rate))
            print(f"Patient {self.patient_id_counter} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_elective_surgery.normal(self.experiment.elective_surgery_arrival_rate, self.experiment.elective_surgery_arrival_std_dev))
            print(f"Elective Patient {self.patient_id_counter} arrived at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.elective_admissions_process())

    def unplanned_admissions_process(self, source):
        patient_id = self.patient_id_counter
        print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")
            # Determine length of stay based on source
            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.experiment.accident_emergency_lognormal_mu, self.experiment.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.experiment.wards_lognormal_mu, self.experiment.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.experiment.emergency_surgery_lognormal_mu, self.experiment.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.experiment.other_hospitals_lognormal_mu, self.experiment.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else:  # X-Ray Department
                mu, sigma = self.lognormal_to_normal(self.experiment.xray_department_lognormal_mu, self.experiment.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)
            yield self.env.timeout(length_of_stay)
            print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")
            # Intensive cleaning
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self):
        patient_id = self.patient_id_counter
        print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
            self.cancelled_operations += 1
        else:
            with self.critical_care_beds.request() as req:
                yield req
                print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
                length_of_stay = self.rng_elective_surgery.exponential(self.experiment.elective_surgery_treatment_rate)
                yield self.env.timeout(length_of_stay)
                print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")
                # Intensive cleaning
                yield self.env.timeout(self.experiment.intensive_cleaning_duration)
                print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())
        self.env.run(until=self.experiment.total_run_length)
        print(f"Total cancelled elective operations: {self.cancelled_operations}")

# Run the simulation
experiment = Experiment()
env = simpy.Environment()
ccu = CCU(env, experiment)
ccu.run()


Patient 0 arrived from the Wards at 4.30 hours
Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 1 arrived from the Wards at 6.54 hours
Patient 2 from Wards requests a critical care bed at 6.54 hours
Patient 2 from Wards admitted to a critical care bed at 6.54 hours
Patient 2 arrived from Emergency Surgery at 13.71 hours
Patient 3 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 3 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Elective Patient 3 arrived at 15.80 hours
Elective Patient 4 requests a critical care bed at 15.80 hours
Elective Patient 4 admitted to a critical care bed at 15.80 hours
Patient 2 from Wards discharged from a critical care bed at 17.12 hours
Intensive cleaning completed for Patient 2 from Wards at 22.12 hours
Patient 4 arrived from Emergency Surgery at 31.66 hours
Patient 5 from Emergency Surgery requests a critical care bed a

## Testing

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class. We also update the test of run length splitting into two tests varying a results collection period and warm-up period.

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [6]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [7]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCU(env, experiment)
    ccu_model.run()
    return ccu_model

In [8]:
def extreme_value_test_1(extreme_value=M*5):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time
    set to $M$ a very large number
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    '''
    experiment = Experiment(wards_arrival_rate = extreme_value,
                            emergency_surgery_arrival_rate = extreme_value, other_hospitals_arrival_rate = extreme_value,
                            xray_department_arrival_rate = extreme_value, elective_surgery_arrival_rate = extreme_value)
    run_test(experiment)

In [9]:
extreme_value_test_1()

Patient 0 arrived from Accident and Emergency at 54.62 hours
Patient 1 from Accident and Emergency requests a critical care bed at 54.62 hours
Patient 1 from Accident and Emergency admitted to a critical care bed at 54.62 hours
Patient 1 arrived from Accident and Emergency at 107.70 hours
Patient 2 from Accident and Emergency requests a critical care bed at 107.70 hours
Patient 2 from Accident and Emergency admitted to a critical care bed at 107.70 hours
Patient 2 from Accident and Emergency discharged from a critical care bed at 112.19 hours
Patient 2 arrived from Accident and Emergency at 114.06 hours
Patient 3 from Accident and Emergency requests a critical care bed at 114.06 hours
Patient 3 from Accident and Emergency admitted to a critical care bed at 114.06 hours
Intensive cleaning completed for Patient 2 from Accident and Emergency at 117.19 hours
Patient 3 arrived from Accident and Emergency at 147.06 hours
Patient 4 from Accident and Emergency requests a critical care bed at 1

### Test 2: Extreme value test 2

PASS

In [10]:
def extreme_value_test_2(extreme_value=M*5):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    experiment = Experiment(accident_emergency_arrival_rate = extreme_value, wards_arrival_rate = extreme_value,
                            emergency_surgery_arrival_rate = extreme_value, other_hospitals_arrival_rate = extreme_value,
                            xray_department_arrival_rate = extreme_value)
    run_test(experiment)

In [11]:
extreme_value_test_2()

Elective Patient 0 arrived at 15.80 hours
Elective Patient 1 requests a critical care bed at 15.80 hours
Elective Patient 1 admitted to a critical care bed at 15.80 hours
Elective Patient 1 arrived at 34.02 hours
Elective Patient 2 requests a critical care bed at 34.02 hours
Elective Patient 2 admitted to a critical care bed at 34.02 hours
Elective Patient 2 arrived at 45.80 hours
Elective Patient 3 requests a critical care bed at 45.80 hours
Elective Patient 3 admitted to a critical care bed at 45.80 hours
Elective Patient 3 arrived at 61.18 hours
Elective Patient 4 requests a critical care bed at 61.18 hours
Elective Patient 4 admitted to a critical care bed at 61.18 hours
Elective Patient 3 discharged from a critical care bed at 61.98 hours
Intensive cleaning completed for Elective Patient 3 at 66.98 hours
Elective Patient 4 arrived at 77.65 hours
Elective Patient 5 requests a critical care bed at 77.65 hours
Elective Patient 5 admitted to a critical care bed at 77.65 hours
Elective

### Test 3: Different run length

PASS

In [12]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model = run_test(experiment)
    return model.env.now

In [13]:
test_results_collection_period(10)

Patient 0 arrived from the Wards at 4.30 hours
Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 1 arrived from the Wards at 6.54 hours
Patient 2 from Wards requests a critical care bed at 6.54 hours
Patient 2 from Wards admitted to a critical care bed at 6.54 hours
Patient 2 arrived from Emergency Surgery at 13.71 hours
Patient 3 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 3 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Elective Patient 3 arrived at 15.80 hours
Elective Patient 4 requests a critical care bed at 15.80 hours
Elective Patient 4 admitted to a critical care bed at 15.80 hours
Patient 2 from Wards discharged from a critical care bed at 17.12 hours
Intensive cleaning completed for Patient 2 from Wards at 22.12 hours
Patient 4 arrived from Emergency Surgery at 31.66 hours
Patient 5 from Emergency Surgery requests a critical care bed a

730

### Test 4: Extreme value test 3

PASS.

In [16]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)

In [17]:
extreme_value_test_3()

Patient 1 arrived from Emergency surgery at time 2.510009074608789
Patient 1 admitted to critical care bed at time 2.510009074608789
Patient 2 arrived from Accident and Emergency at time 9.113211670063446
Patient 3 arrived from Emergency surgery at time 11.527201522444608
Elective surgery patient 4 arrived at time 15.598945320419698
Elective surgery for patient 4 cancelled due to no available critical care beds at time 15.598945320419698
Patient 5 arrived from Accident and Emergency at time 16.75658717917721
Patient 6 arrived from Accident and Emergency at time 17.800350215421926
Patient 6 discharged from critical care bed at time 17.931927811135694
Critical care bed is available for next patient at time 22.931927811135694
Patient 6 admitted to critical care bed at time 22.931927811135694
Elective surgery patient 7 arrived at time 28.68783092403146
Elective surgery for patient 7 cancelled due to no available critical care beds at time 28.68783092403146
Patient 8 arrived from Accident a

### Test 5: Extreme value test 4

PASS

In [14]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    experiment = Experiment(accident_emergency_lognormal_mu=M, 
                 wards_lognormal_mu=M, 
                 emergency_surgery_lognormal_mu=M, 
                 other_hospitals_lognormal_mu=M, 
                 xray_department_lognormal_mu=M, 
                 elective_surgery_treatment_rate=M)
    run_test(experiment)

In [15]:
extreme_value_test_4()

Patient 0 arrived from the Wards at 4.30 hours
Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 1 arrived from the Wards at 6.54 hours
Patient 2 from Wards requests a critical care bed at 6.54 hours
Patient 2 from Wards admitted to a critical care bed at 6.54 hours
Patient 2 arrived from Emergency Surgery at 13.71 hours
Patient 3 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 3 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Elective Patient 3 arrived at 15.80 hours
Elective Patient 4 requests a critical care bed at 15.80 hours
Elective Patient 4 admitted to a critical care bed at 15.80 hours
Patient 4 arrived from Emergency Surgery at 31.66 hours
Patient 5 from Emergency Surgery requests a critical care bed at 31.66 hours
Patient 5 from Emergency Surgery admitted to a critical care bed at 31.66 hours
Elective Patient 5 arrived at 34.02 hours
Elect

### Test 6: Vary Warm-up parameter

PASS

In [16]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [17]:
test_warmup_period(0)

Patient 0 arrived from the Wards at 4.30 hours
Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 1 arrived from the Wards at 6.54 hours
Patient 2 from Wards requests a critical care bed at 6.54 hours
Patient 2 from Wards admitted to a critical care bed at 6.54 hours
Patient 2 arrived from Emergency Surgery at 13.71 hours
Patient 3 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 3 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Elective Patient 3 arrived at 15.80 hours
Elective Patient 4 requests a critical care bed at 15.80 hours
Elective Patient 4 admitted to a critical care bed at 15.80 hours
Patient 2 from Wards discharged from a critical care bed at 17.12 hours
Intensive cleaning completed for Patient 2 from Wards at 22.12 hours
Patient 4 arrived from Emergency Surgery at 31.66 hours
Patient 5 from Emergency Surgery requests a critical care bed a

(8640, 8640)